### 2do Cuatrimestre 2018 2da Oportunidad

In [197]:
from datetime import datetime
import datetime as dt
import pandas as pd

# armamos un data frame
sales_data = {
        'Dia': [7,20,15,17,18, 16, 3, 4],
        'Mes': [1,2,3,3,3, 3, 4, 4], 
        'Año': [2019, 2019, 2020, 2020, 2020, 2020, 2020, 2020],
        'ProductID': ['PID_1','PID_1','PID_2','PID_2','PID_2', 'PID_2', 'PID_3', 'PID_3'],
        'Importe(USD)': [10,15,20,20,30, 31, 150, 40]
}

product_data = {
        'ProductId': ['PID_1','PID_2','PID_3','PID_4','PID_5'],
        'Title': ['Title_1','Title_2','Title_3','Title_4','Title_5'],
        'Category': ['Men','Women','Men','Kids','Men'],
        'Description': ['Des_1','Des_2','Des_3','Des_4','Des_5']
}

sales = pd.DataFrame(sales_data, columns = sales_data.keys())
products = pd.DataFrame(product_data, columns = product_data.keys())

In [198]:
sales.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
Dia             8 non-null int64
Mes             8 non-null int64
Año             8 non-null int64
ProductID       8 non-null object
Importe(USD)    8 non-null int64
dtypes: int64(4), object(1)
memory usage: 448.0+ bytes


In [199]:
products

,ProductId,Title,Category,Description
0,PID_1,Title_1,Men,Des_1
1,PID_2,Title_2,Women,Des_2
2,PID_3,Title_3,Men,Des_3
3,PID_4,Title_4,Kids,Des_4
4,PID_5,Title_5,Men,Des_5


In [200]:
sales_filtered = sales.loc[:,['ProductID','Mes','Importe(USD)']]
products_filtered = products.loc[:,['ProductId','Category','Title']]

In [201]:
df_merge = sales_filtered.merge(products_filtered,left_on = 'ProductID',right_on='ProductId')
df_merge

,ProductID,Mes,Importe(USD),ProductId,Category,Title
0,PID_1,1,10,PID_1,Men,Title_1
1,PID_1,2,15,PID_1,Men,Title_1
2,PID_2,3,20,PID_2,Women,Title_2
3,PID_2,3,20,PID_2,Women,Title_2
4,PID_2,3,30,PID_2,Women,Title_2
5,PID_2,3,31,PID_2,Women,Title_2
6,PID_3,4,150,PID_3,Men,Title_3
7,PID_3,4,40,PID_3,Men,Title_3


In [202]:
df_merge_men = df_merge.loc[df_merge['Category'] == 'Men',:]
df_merge_men['PromedioMensual'] = df_merge_men.groupby('Mes')['Importe(USD)'].transform('mean') 
df_merge_men


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ProductID,Mes,Importe(USD),ProductId,Category,Title,PromedioMensual
0,PID_1,1,10,PID_1,Men,Title_1,10
1,PID_1,2,15,PID_1,Men,Title_1,15
6,PID_3,4,150,PID_3,Men,Title_3,95
7,PID_3,4,40,PID_3,Men,Title_3,95


In [203]:
df_merge_men.loc[df_merge_men['Importe(USD)'] > df_merge_men['PromedioMensual'],:]['Title']

6    Title_3
Name: Title, dtype: object

In [204]:
sales

,Dia,Mes,Año,ProductID,Importe(USD)
0,7,1,2019,PID_1,10
1,20,2,2019,PID_1,15
2,15,3,2020,PID_2,20
3,17,3,2020,PID_2,20
4,18,3,2020,PID_2,30
5,16,3,2020,PID_2,31
6,3,4,2020,PID_3,150
7,4,4,2020,PID_3,40


In [231]:
sales['Fecha'] = pd.to_datetime(sales['Año'].astype('str') +'-'+sales['Mes'].astype('str')+'-'+sales['Dia'].astype('str'))
sales['FechaDays'] = sales['Fecha'] - pd.to_datetime('2019-01-01',format = '%Y-%m-%d')
sales_filter = sales.loc[:,['ProductID','Fecha','FechaDays']].sort_values(by = ['FechaDays'])
sales_filter['DiasConsecutivos'] = sales_filter.groupby('ProductID').agg({'FechaDays':'diff'}) == dt.timedelta(days = 1)
sales_filter.groupby('ProductID').agg({'DiasConsecutivos':'sum'}).nlargest(10, "DiasConsecutivos")


,DiasConsecutivos
ProductID,
PID_2,3.0
PID_3,1.0
PID_1,0.0
